# FIND
`find` recorre un árbol de directorios y evalúa cada archivo contra reglas que tú defines.

  * No "busca texto"
  * No "mira solo nombres"
  * Camina el sistema de archivos y toma de decisiones

## Sintaxis
```bash
find [dónde-empiezo] [criterio] [acciones]
```

> Empieza aquí → entra a cada carpeta → para cada archivo, pregúntate cosas → si cumple, haz algo

### Uso simple

In [9]:
!find .

.
./1-find.ipynb


  * `find` entra recursivamente en todas las carpetas
  * `.` = directorio actual

## Criterios

### Nombre | `-name`
Compara nombre si coincide lo muestra

In [17]:
!find . -name 1-find.ipynb

./1-find.ipynb


In [20]:
!find . -iname 1-FIND.ipynb # Ignora mayúscula 

./1-find.ipynb


In [22]:
!find . -name "*.ipynb" # Usando *

./1-find.ipynb


> Todo patrón para find va entre comillas

### Tipo | `-type`

Especificar por tipo:
  * `f` -> archivo
  * `d` -> directorio
  * `l` -> enlace simbólico

In [23]:
!find ../ -type d 

../
../level-1
../level-2
../level-3


In [26]:
!find . -type f

./1-find.ipynb


### Usuario | `-user`
Encontrar por  propietario de archivo

In [31]:
!whoami # Mi usuario
!ls -l 
!find . -type f -user user # Muestra solo archivos de este usuario

user


total 8
-rw-r--r-- 1 user user 4800 Jan 17 04:18 1-find.ipynb
./1-find.ipynb


### Grupo | `-group`
Encontrar por grupo del archivo

In [37]:
!groups # Ver grupos de usuario actual
!ls -l # Archivo de grupo user
!find . -type f -group user

user dialout cdrom floppy sudo audio dip video plugdev netdev lpadmin bluetooth scanner


total 8
-rw-r--r-- 1 user user 5536 Jan 17 04:27 1-find.ipynb
./1-find.ipynb


### Tamaño | `-size`
Por tamaño específico
  * `c` -> bytes
  * `k` -> kilobytes
  * `M` -> megabytes
  * `G` -> gigabytes

Comparadores: 
  * `+` -> mayor que
  * `-` -> menor que 
  * Sin signo -> exactamente 


In [40]:
!ls -lh ../../files/test/

total 28K
-rw-r--r-- 1 user user 3.9K Jan 14 03:45 access.logs
-rw-r--r-- 1 user user 4.4K Jan 14 03:45 advanced_attacks.logs
-rw-r--r-- 1 user user 3.1K Jan 14 03:45 archivo-grande.txt
-rw-r--r-- 1 user user 1.3K Jan 14 03:45 archivos-datos.txt
-rw-r--r-- 1 user user   50 Jan 14 03:45 prueba.txt
-rw-r--r-- 1 user user  640 Jan 17 02:52 test.txt


In [41]:
!find ../../files/test/ -type f -size +4k

../../files/test/advanced_attacks.logs


### SUID | 4000
Con archivos SUID con `-perm`

In [44]:
!find . -perm 4000 2>/dev/null

> Dirigir `STDERR` para evitar mezclar son `STDOUT` 

### SGID | 2000
Con archivos SGID con `-perm`

In [45]:
!find . -perm 2000 2>/dev/null

### Sticky Bit | 1000
Con archivos Sticky con `-perm`

In [47]:
!find . -perm 1000 2>/dev/null

### Octal | XYZ
Buscar con permisos específicos 755, 600 ...

In [1]:
!find . -perm 755 2>/dev/null

.


### Otros
Ejecutables y escribibles 

In [2]:
!find . -type f -executable -writable   

## Acción
Algo que se ejecuta cuando un archivo pasa todos los filtros.

###  `-print` | Default
Acción por defecto

> Si un archivo no cumple las condiciones, la acción nunca se ejecuta.

#### Explícito
```bash
!find . -type f -print
```
  * Escribe la ruta completa en `STDOUT`
  * Añade un `\n` al final
  * No formatea ni escapa nada

#### Implícito 
```bash
!find . # Interno equivale a find . -print
```
> Si usas cualquier otra acción, -print deja de ejecutarse automáticamente.

In [22]:
!find ../../files/test/ -type f -print # Salto de línea \n

../../files/test/access.logs
../../files/test/advanced_attacks.logs
../../files/test/archivo-grande.txt
../../files/test/archivos-datos.txt
../../files/test/prueba.txt
../../files/test/test.txt


### `-print0`
```bash
find . -type f -print0
```
  * Usa `\0`(NULL) en vez de `\n`
  * Diseñado para pipes seguros (`xargs -0`)


In [26]:
!find ../../files/test/ -type f -print0 # Muestra en una misma línea separado por \0 no visible

../../files/test/access.logs ../../files/test/advanced_attacks.logs ../../files/test/archivo-grande.txt ../../files/test/archivos-datos.txt ../../files/test/prueba.txt ../../files/test/test.txt 

#### Usado con `xargs -0`
Con `-print` lo rompe (\n) y con `-print0` no, por eso trabaja con `xargs -0`

> Recordar que `xargs` convierte una entrada por stdin en argumentos de un comando.

**¿Qué es `-0`?**
Los argumentos no están separados por espacios ni líneas sino por el byte NULL(`\0`)


```bash
find . -type f -print0 | xargs -0 rm
```

  * `xargs -0` lee hasta `\0`
  * Crea argumentos exactos
  * Ejecuta `rm`: rm file1 file2 file3 ...

In [29]:
!find ../../files/test/ -type f -print0 | xargs -0 ls -l

-rw-r--r-- 1 user user 3979 Jan 14 03:45 ../../files/test/access.logs
-rw-r--r-- 1 user user 4413 Jan 14 03:45 ../../files/test/advanced_attacks.logs
-rw-r--r-- 1 user user 3143 Jan 14 03:45 ../../files/test/archivo-grande.txt
-rw-r--r-- 1 user user 1265 Jan 14 03:45 ../../files/test/archivos-datos.txt
-rw-r--r-- 1 user user   50 Jan 14 03:45 ../../files/test/prueba.txt
-rw-r--r-- 1 user user  640 Jan 17 02:52 ../../files/test/test.txt


### `-exec` | Acción estrella

#### a. Modo simple `-exec ... \`
```bash
find . -type f -name "*.log" -exec rm {} \;
```
Por cada archivo:
  * Se sustituye `{}` por la ruta 
  * Se lanza un proceso `rm`

Lo que hace internamente:
```ouput
rm ./a.log
rm ./b.log
rm ./c.log
```

In [32]:
# LABORATORIO
!mkdir ../../files/borrador/tmp

In [35]:
# Agregando archivos
!touch ../../files/borrador/tmp/texto{1..9}.txt && ls ../../files/borrador/tmp

texto1.txt  texto3.txt	texto5.txt  texto7.txt	texto9.txt
texto2.txt  texto4.txt	texto6.txt  texto8.txt


In [37]:
# Usando find
!find ../../files/borrador/tmp -type f -name "*.txt" -exec rm {} \;

In [38]:
# Verificando
!ls ../../files/borrador/tmp

#### b. Modo eficiente `-exec ..+` 
```bash
find . -type f -name "*.log" -exec rm {} +
```
Internamente: `rm ./a.log ./b.log ./c.log`

Menos procesos, más rápido y seguro.

> Usa + salvo que necesites una ejecución por archivo.

### `-execdir`  | Seguridad avanzada
```bash
find . -type f -execdir rm {} \;
```
Diferencia clave
  * Ejecuta el comando dentro del directorio del archivo
  * {} es solo el nombre, no lo ruta

Por qué existe:
  * Evita ataques de race condition
  * Más seguro con directorios world-writable

#### Laboratorio
##### a. Diferencia
| Aspecto                   | -exec               | -execdir               |
| ------------------------- | ------------------- | ---------------------- |
| Directorio de ejecución   | donde lanzas `find` | directorio del archivo |
| `{}`                      | ruta completa       | solo nombre            |
| Riesgo con dirs inseguros | alto                | bajo                   |
| Uso como root             | peligroso           | recomendado            |
| Performance               | similar             | similar                |





##### b. Visual
Ver como funciona ambos 

In [42]:
!find ../../files/ -type f -name "test.txt" -exec echo {} \; # Muestra ruta completa

../../files/test/test.txt


  * Ejecuta el comando desde donde eestás 
  * Usa rutas completas

In [43]:
!find ../../files/ -type f -name "test.txt" -execdir echo {} \; # Solo nombre 

./test.txt


  * Entra al directorio del archivo
  * Ejecuta el comando ahí
  * Es más seguro

Ejemplo: `find . -type f -name "archivo.txt" -execdir rm {} \;`


`-execdir` dice:
  > “No borres el archivo desde donde estás.
Muévete primero a la habitación del archivo
y luego ejecútalo ahí.”

### `-delete` | Acción delete
Elimina exactamente el archivo que `find` encontró, no algo que cambió después.

> TOCTOU:  es un vulnerabilidad de seguridad en sistemas Linux que surge de a condición de carrera cuando un programa verifica el estado de un archivo (por ejemplo, permisos) y luego usa ese archivo, con una ventana intermedia donde el estado del archivo puede cambiar.

```bash
find . -type f -name "*.tmp" -delete
```

  * Recorre el árbol
  * Identifica un archivo
  * **Lo elimina internamente**
  * **SIN lanzar un comando externo**

No hay:
  * `rm`
  * shell
  * expansión
  * cambio de contexto

**No existe ventana TOCTOU**    
